Supongamos que haces una pregunta compleja a miles de personas al azar y luego agregas sus respuestas. En muchos casos, encontrará que esta respuesta agregada es mejor que la respuesta de un experto. Esto se llama la sabiduría de la multitud. Del mismo modo, si agrega las predicciones de un grupo de predictores (como clasificadores o regresiónres), a menudo obtendrá mejores predicciones que con el mejor predictor individual. Un grupo de predictores se llama conjunto; por lo tanto, esta técnica se llama aprendizaje conjunto, y un algoritmo de aprendizaje conjunto se llama método conjunto.

Como ejemplo de un método de conjunto, puedes entrenar a un grupo de clasificadores de árbol de decisiones, cada uno en un subconjunto aleatorio diferente del conjunto de entrenamiento. A continuación, puede obtener las predicciones de todos los árboles individuales, y la clase que obtiene la mayor cantidad de votos es la predicción del conjunto (ver el último ejercicio en el capítulo 6). Tal conjunto de árboles de decisión se llama bosque aleatorio, y a pesar de su simplicidad, este es uno de los algoritmos de aprendizaje automático más potentes disponibles en la actualidad.

Como se discutió en el capítulo 2, a menudo usará métodos de conjunto cerca del final de un proyecto, una vez que ya haya construido algunos buenos predictores, para combinarlos en un predictor aún mejor. De hecho, las soluciones ganadoras en los concursos de aprendizaje automático a menudo implican varios métodos de conjunto, los más famosos en el concurso de premios Netflix.

En este capítulo examinaremos los métodos de conjunto más populares, incluidos los clasificadores de votación, los conjuntos de embolsado y pegado, los bosques aleatorios y los conjuntos de refuerzo y apilamiento.

# Clasificadores de votación

Supongamos que has entrenado a algunos clasificadores, cada uno de los cuales alcanza una precisión de alrededor del 80 %. Puede tener un clasificador de regresión logística, un clasificador SVM, un clasificador de bosques aleatorio, un clasificador de vecinos más cercanos y tal vez algunos más (ver Figura 7-1).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0701.png)

(_Figura 7-1. Formación de diversos clasificadores_)

Una forma muy sencilla de crear un clasificador aún mejor es agregar las predicciones de cada clasificador: la clase que obtiene la mayor cantidad de votos es la predicción del conjunto. Este clasificador de voto mayoritario se llama clasificador de voto duro (ver Figura 7-2).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0702.png)

(_Figura 7-2. Predicciones de clasificación de votación difícil_)

Sorprendentemente, este clasificador de votación a menudo logra una mayor precisión que el mejor clasificador del conjunto. De hecho, incluso si cada clasificador es un alumno débil (lo que significa que solo lo hace un poco mejor que la adivinación aleatoria), el conjunto puede seguir siendo un aprendiz fuerte (lo que alcanza una alta precisión), siempre que haya un número suficiente de alumnos débiles en el conjunto y sean lo suficientemente diversos.

¿Cómo es esto posible? La siguiente analogía puede ayudar a arrojar algo de luz sobre este misterio. Supongamos que tiene una moneda ligeramente sesgada que tiene un 51 % de probabilidades de subir cabezas y un 49 % de probabilidades de subir colas. Si lo lanzas 1.000 veces, generalmente obtendrás más o menos 510 cabezas y 490 colas, y por lo tanto una mayoría de cabezas. Si haces los cálculos, encontrarás que la probabilidad de obtener la mayoría de las cabezas después de 1000 lanzamientos es cercana al 75 %. Cuanto más lances la moneda, mayor será la probabilidad (por ejemplo, con 10.000 lanzamientos, la probabilidad aumenta por encima del 97 %). Esto se debe a la ley de los grandes números: a medida que sigues lanzando la moneda, la proporción de cabezas se acerca cada vez más a la probabilidad de cabezas (51%). La figura 7-3 muestra 10 series de lanzamientos de monedas sesgadas. Puedes ver que a medida que aumenta el número de lanzamientos, la proporción de cabezas se acerca al 51 %. Eventualmente, las 10 series terminan tan cerca del 51 % que están constantemente por encima del 50 %.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0703.png)

Del mismo modo, supongamos que construyes un conjunto que contiene 1000 clasificadores que son individualmente correctos solo el 51 % del tiempo (apenas mejor que la adivinanza aleatoria). Si predices la clase votada por la mayoría, ¡puedes esperar una precisión de hasta un 75 %! Sin embargo, esto solo es cierto si todos los clasificadores son perfectamente independientes, lo que produce errores no correlacionados, lo que claramente no es el caso porque están entrenados con los mismos datos. Es probable que cometan el mismo tipo de errores, por lo que habrá muchos votos mayoritarios para la clase equivocada, lo que reducirá la precisión del conjunto.


TIP:

- Los métodos de conjunto funcionan mejor cuando los predictores son lo más independientes posible entre sí. Una forma de obtener clasificadores diversos es entrenarlos utilizando algoritmos muy diferentes. Esto aumenta la posibilidad de que cometan tipos de errores muy diferentes, mejorando la precisión del conjunto.

Scikit-Learn proporciona una clase VotingClassifier que es bastante fácil de usar: simplemente dale una lista de pares de nombre/predictor y úsala como un clasificador normal. Probémoslo en el conjunto de datos de las lunas (presentado en el Capítulo 5). Cargaremos y dividiremos el conjunto de datos de las lunas en un conjunto de entrenamiento y un conjunto de prueba, luego crearemos y entrenaremos un clasificador de votación compuesto por tres clasificadores diversos:

In [23]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

voting_clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(random_state=42))
    ]
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

Cuando ajustas un `VotingClassifier`, clona cada estimador y ajusta los clones. Los estimadores originales están disponibles a través del atributo `estimator`, mientras que los clones ajustados están disponibles a través del atributo `estimators_`. 
Si prefieres un dictado en lugar de una lista, puedes usar `named_estimators` o `named_estimators_` en su lugar. Para comenzar, veamos la precisión de cada clasificador ajustado en el conjunto de prueba:

In [24]:
for name, clf in voting_clf.named_estimators_.items():
    print(name, "=", clf.score(X_test, y_test))

lr = 0.864
rf = 0.896
svc = 0.896


Cuando llama al método `predict()` del clasificador de votación, realiza una votación estricta. Por ejemplo, el clasificador de votación predice la clase 1 para la primera instancia del conjunto de prueba, porque dos de cada tres clasificadores predicen esa clase:

In [25]:
voting_clf.predict(X_test[:1])

array([1])

In [26]:
[clf.predict(X_test[:1]) for clf in voting_clf.estimators_]

[array([1]), array([1]), array([0])]

Ahora veamos el rendimiento del clasificador de votación en el conjunto de pruebas:

In [27]:
voting_clf.score(X_test, y_test)

0.912

¡Ahí lo tienes! El clasificador de votación supera a todos los clasificadores individuales.

Si todos los clasificadores pueden estimar las probabilidades de clase (es decir, si todos tienen un método `predict_proba()`), entonces puede decirle a Scikit-Learn que prediga la clase con la probabilidad de clase más alta, promediada entre todos los clasificadores individuales. 

A esto se le llama soft voting. 

A menudo logra un mayor rendimiento que la votación dura porque da más peso a los votos con mucha confianza. Todo lo que necesita hacer es configurar el hiperparámetro de votación del clasificador de `voting` en "soft" y asegurarse de que todos los clasificadores puedan estimar las probabilidades de clase. Este no es el caso de la clase SVC de forma predeterminada, por lo que debe establecer su hiperparámetro de `probability` en `True` (esto hará que la clase SVC use validación cruzada para estimar las probabilidades de la clase, lo que ralentizará el entrenamiento y agregará un `predict_proba()` método). Probemos eso:

In [28]:
voting_clf.voting = "soft"
voting_clf.named_estimators["svc"].probability = True
voting_clf.fit(X_train, y_train)
voting_clf.score(X_test, y_test)

0.92

Llegamos al 92 % de precisión simplemente usando el voto suave, ¡no está mal!

## Bagging (embolsado) y Taping (pegado)


Una forma de obtener un conjunto diverso de clasificadores es utilizar algoritmos de entrenamiento muy diferentes, como se acaba de discutir. Otro enfoque es utilizar el mismo algoritmo de entrenamiento para cada predictor, pero entrenarlos en diferentes subconjuntos aleatorios del conjunto de entrenamiento. Cuando el muestreo se realiza con reemplazo, ⁠1 este método se llama embolsado⁠2 (abreviatura de agregación de arranque ⁠3). Cuando el muestreo se realiza sin reemplazo, se llama pegado.⁠4

En otras palabras, tanto el embolsado como el pegado permiten que las instancias de entrenamiento se muestreen varias veces a través de múltiples predictores, pero solo el embolsado permite que las instancias de entrenamiento se muestreen varias veces para el mismo predictor. Este proceso de muestreo y entrenamiento se muestra en la Figura 7-4.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0704.png)

(_Figura 7-4. El embolsado y pegado implica entrenar a varios predictores en diferentes muestras aleatorias del conjunto de entrenamiento_)

Una vez que se entrenan todos los predictores, el conjunto puede hacer una predicción para una nueva instancia simplemente agregando las predicciones de todos los predictores. La función de agregación suele ser el modo estadístico para la clasificación (es decir, la predicción más frecuente, al igual que con un clasificador de votación dura), o el promedio para la regresión. Cada predictor individual tiene un sesgo más alto que si estuviera entrenado en el conjunto de entrenamiento original, pero la agregación reduce tanto el sesgo como la varianza.⁠5 En general, el resultado neto es que el conjunto tiene un sesgo similar pero una varianza más baja que un solo predictor entrenado en el conjunto de entrenamiento original.

Como se puede ver en la Figura 7-4, todos los predictores se pueden entrenar en paralelo, a través de diferentes núcleos de CPU o incluso diferentes servidores. Del mismo modo, las predicciones se pueden hacer en paralelo. Esta es una de las razones por las que el ensacar y pegar son métodos tan populares: escalan muy bien.


## Bagging y Taping en Scikit-Learn


Scikit-Learn ofrece una API simple para empaquetar y pegar: clase `BaggingClassifier` (o `BaggingRegressor` para regresión). 
El siguiente código entrena un conjunto de 500 clasificadores de árboles de decisión:⁠6 cada uno se entrena en 100 instancias de entrenamiento tomadas aleatoriamente del conjunto de entrenamiento con reemplazo (este es un ejemplo de embolsado, pero si desea utilizar el pegado en su lugar, simplemente configure `bootstrap= False`). El parámetro `n_jobs` le dice a Scikit-Learn la cantidad de núcleos de CPU que se usarán para el entrenamiento y las predicciones, y `–1` le dice a Scikit-Learn que use todos los núcleos disponibles:

In [29]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            max_samples=100, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1, random_state=42)

La figura 7-5 compara el límite de decisión de un solo árbol de decisión con el límite de decisión de un conjunto de bolsas de 500 árboles (del código anterior), ambos entrenados en el conjunto de datos de lunas. Como puede ver, las predicciones del conjunto probablemente se generalizarán mucho mejor que las predicciones del árbol de decisión único: el conjunto tiene un sesgo comparable pero una varianza más pequeña (hace aproximadamente el mismo número de errores en el conjunto de entrenamiento, pero el límite de decisión es menos irregular).

El embolsado introduce un poco más de diversidad en los subconjuntos en los que se entrena cada predictor, por lo que el embolsado termina con un sesgo ligeramente más alto que el pegado; pero la diversidad adicional también significa que los predictores terminan estando menos correlacionados, por lo que la varianza del conjunto se reduce. En general, el embolsado a menudo da como resultado mejores modelos, lo que explica por qué generalmente se prefiere. Pero si tienes tiempo libre y potencia de la CPU, puedes usar la validación cruzada para evaluar tanto el embolsado como el pegado y seleccionar el que mejor funcione.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0705.png)

(_Figura 7-5. Un solo árbol de decisión (izquierda) frente a un conjunto de embolsado de 500 árboles (derecha)_)



## Evaluación fuera del Bagging

Con el embolsado, es posible que algunas instancias de entrenamiento se muestreen varias veces para cualquier predictor determinado, mientras que es posible que otras no se muestreen en absoluto. 
De forma predeterminada, un `BaggingClassifier` muestra m instancias de entrenamiento con reemplazo (`bootstrap=True`), donde m es el tamaño del conjunto de entrenamiento. Con este proceso, se puede demostrar matemáticamente que solo alrededor del 63 % de las instancias de entrenamiento se muestrean en promedio para cada predictor.⁠7 El 37 % restante de las instancias de entrenamiento que no se muestrean se denominan listas para usar (Out-Of-Bag). instancias. 

Tenga en cuenta que no son los mismos 37% para todos los predictores.

Un conjunto de embolsado se puede evaluar utilizando instancias de OOB, sin la necesidad de un conjunto de validación separado: de hecho, si hay suficientes estimadores, entonces cada instancia en el conjunto de entrenamiento probablemente será una instancia de OOB de varios estimadores, por lo que estos estimadores se pueden utilizar para hacer una predicción de conjunto justa para esa instancia. Una vez que tenga una predicción para cada instancia, puede calcular la precisión de la predicción del conjunto (o cualquier otra métrica).

En Scikit-Learn, puede configurar `oob_score=True` al crear un `BaggingClassifier` para solicitar una evaluación OOB automática después del entrenamiento. El siguiente código demuestra esto. La puntuación de evaluación resultante está disponible en el atributo `oob_score_`:

In [30]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, oob_score=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.896

Según esta evaluación OOB, es probable que este BaggingClassifier logre aproximadamente un 89,6% de precisión en el conjunto de prueba. 

Verifiquemos esto:

In [31]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.92

Tenemos una precisión del 92 % en la prueba. La evaluación de la OOB fue un poco demasiado pesimista, un poco más del 2 % demasiado baja.

La función de decisión OOB para cada instancia de entrenamiento también está disponible a través del atributo de  `oob_decision_function_`. Dado que el estimador base tiene el método `predict_proba()`, la función de decisión devuelve las probabilidades de la clase para cada instancia de entrenamiento. Por ejemplo, la evaluación de la OOB estima que la primera instancia de entrenamiento tiene una probabilidad del 63,6 % de pertenecer a la clase positiva y una probabilidad del 32,4 % de pertenecer a la clase negativa:

In [32]:
bag_clf.oob_decision_function_[:3]  # probas para las primeras 3 instancias

array([[0.32352941, 0.67647059],
       [0.3375    , 0.6625    ],
       [1.        , 0.        ]])

## Parches aleatorios y subespacios aleatorios

La clase `BaggingClassifier` también admite el muestreo de características. El muestreo está controlado por dos hiperparámetros: `max_features` y `bootstrap_features`. 

Funcionan de la misma manera que `max_samples` y `bootstrap`, pero para muestreo de características en lugar de muestreo de instancias. Por lo tanto, cada predictor se entrenará en un subconjunto aleatorio de características de entrada.

Esta técnica es particularmente útil cuando se trata de entradas de alta dimensión (como imágenes), ya que puede acelerar considerablemente el entrenamiento. 

El muestreo de instancias y características de entrenamiento se denomina método de parches aleatorios.⁠ 

Mantener todas las instancias de entrenamiento (estableciendo `bootstrap=False y max_samples=1.0`) pero muestreando características (estableciendo `bootstrap_features en True` y/o `max_features` en un valor menor que `1.0`) se llama método de subespacios aleatorios.⁠

Las características de muestreo dan como resultado una diversidad de predictores aún mayor, negociando un poco más de sesgo por una varianza más baja.

# Bosques aleatorios



Como hemos comentado, un bosque aleatorio⁠ es un conjunto de árboles de decisión, generalmente entrenados mediante el método de embolsado (o, a veces, pegado), normalmente con `max_samples` establecido en el tamaño del conjunto de entrenamiento. En lugar de crear un `BaggingClassifier` y pasarle un `DecisionTreeClassifier`, puede usar la clase `RandomForestClassifier`, que es más conveniente y está optimizada para árboles de decisión⁠11 (de manera similar, existe una clase `RandomForestRegressor` para tareas de regresión). El siguiente código entrena un clasificador de bosque aleatorio con 500 árboles, cada uno limitado a un máximo de 16 nodos hoja, utilizando todos los núcleos de CPU disponibles:

In [33]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16,
                                 n_jobs=-1, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

Con algunas excepciones, un `RandomForestClassifier` tiene todos los hiperparámetros de un `DecisionTreeClassifier` (para controlar cómo crecen los árboles), además de todos los hiperparámetros de un `BaggingClassifier` para controlar el conjunto en sí.

El algoritmo de bosque aleatorio introduce una aleatoriedad adicional al cultivar árboles; en lugar de buscar la mejor característica al dividir un nodo (ver Capítulo 6), busca la mejor característica entre un subconjunto aleatorio de características. Por defecto, muestra **sqrt(n)**
norte
  características (donde n es el número total de características). El algoritmo da como resultado una mayor diversidad de árboles, lo que (nuevamente) intercambia un mayor sesgo por una menor varianza, lo que generalmente produce un mejor modelo en general. Entonces, el siguiente BaggingClassifier es equivalente al `RandomForestClassifier` anterior:

In [34]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="sqrt", max_leaf_nodes=16),
    n_estimators=500, n_jobs=-1, random_state=42)

## Árboles adicionales

Cuando se cultiva un árbol en un bosque aleatorio, en cada nodo solo se considera para dividir un subconjunto aleatorio de características (como se analizó anteriormente). Es posible hacer que los árboles sean aún más aleatorios utilizando también umbrales aleatorios para cada característica en lugar de buscar los mejores umbrales posibles (como lo hacen los árboles de decisión normales). Para esto, simplemente configure `splitter="random"` al crear un `DecisionTreeClassifier`.

Un bosque de árboles tan extremadamente aleatorios se llama un conjunto de árboles extremadamente aleatorios⁠ (o extraárboles para abreviar). 
Una vez más, esta técnica cambia más sesgo por una variación menor. También hace que los clasificadores de árboles extra sean mucho más rápidos de entrenar que los bosques aleatorios regulares, porque encontrar el mejor umbral posible para cada característica en cada nodo es una de las tareas más lentas para cultivar un árbol.

Puede crear un clasificador de árboles adicionales utilizando la clase `ExtraTreesClassifier` de Scikit-Learn. 
Su API es idéntica a la clase `RandomForestClassifier`, excepto que el valor predeterminado de `bootstrap` es `False`. 
De manera similar, la clase `ExtraTreesRegressor` tiene la misma API que la clase `RandomForestRegressor`, excepto que el valor predeterminado de `bootstrap` es `False`.

#### TIP

Es difícil saber de antemano si `RandomForestClassifier` funcionará mejor o peor que `ExtraTreesClassifier`. Generalmente, la única forma de saberlo es probar ambos y compararlos mediante validación cruzada.


## Importancia de la característica

Otra gran cualidad de los bosques aleatorios es que facilitan la medición de la importancia relativa de cada característica. Scikit-Learn mide la importancia de una característica mirando hasta qué medida los nodos de los árboles que usan esa característica reducen la impureza en promedio, en todos los árboles del bosque. Más precisamente, es un promedio ponderado, donde el peso de cada nodo es igual al número de muestras de entrenamiento que están asociadas con él (ver Capítulo 6).

Scikit-Learn calcula esta puntuación automáticamente para cada característica después del entrenamiento, luego escala los resultados para que la suma de todas las importancias sea igual a 1. 

Puede acceder al resultado utilizando la variable `feature_importances_`. 

Por ejemplo, el siguiente código entrena un `RandomForestClassifier` en el conjunto de datos del iris (presentado en el Capítulo 4) y genera la importancia de cada característica. Parece que las características más importantes son la longitud del pétalo (44%) y el ancho (42%), mientras que la longitud y el ancho del sépalo son poco importantes en comparación (11% y 2%, respectivamente):

In [35]:
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf.fit(iris.data, iris.target)
for score, name in zip(rnd_clf.feature_importances_, iris.data.columns):
    print(round(score, 2), name)

0.11 sepal length (cm)
0.02 sepal width (cm)
0.44 petal length (cm)
0.42 petal width (cm)


Del mismo modo, si entrena a un clasificador de bosques al azar en el conjunto de datos MNIST (introducido en el capítulo 3) y traza la importancia de cada píxel, obtiene la imagen representada en la Figura 7-6.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0706.png)

(_Figura 7-6. Importancia de los píxeles MNIST (según un clasificador forestal aleatorio)_)


## Boost (impulso)

El boost (originalmente llamado boost de hipótesis) se refiere a cualquier método de conjunto que pueda combinar a varios estudiantes débiles en un alumno fuerte. 

La idea general de la mayoría de los métodos de refuerzo es entrenar a los predictores de forma secuencial, cada uno tratando de corregir a su predecesor. Hay muchos métodos de refuerzo disponibles, pero, con mucho, los más populares son **AdaBoost⁠** (abreviatura de refuerzo adaptativo) y el aumento de gradiente. Empecemos con **AdaBoost**.

### AdaBoost

Una forma de que un nuevo predictor corrija a su predecesor es prestar un poco más de atención a las instancias de entrenamiento que el predecesor no se adapta. Esto da como resultado nuevos predictores que se centran cada vez más en los casos difíciles. Esta es la técnica utilizada por AdaBoost.

Por ejemplo, cuando se entrena un clasificador AdaBoost, el algoritmo primero entrena un clasificador base (como un árbol de decisión) y lo utiliza para hacer predicciones sobre el conjunto de entrenamiento. El algoritmo aumenta entonces el peso relativo de las instancias de entrenamiento mal clasificadas. Luego entrena a un segundo clasificador, utilizando los pesos actualizados, y de nuevo hace predicciones en el conjunto de entrenamiento, actualiza los pesos de la instancia, y así sucede (ver Figura 7-7).

La figura 7-8 muestra los límites de decisión de cinco predictores consecutivos en el conjunto de datos de lunas (en este ejemplo, cada predictor es un clasificador SVM altamente regularizado con un núcleo RBF).⁠ El primer clasificador se equivoca en muchas instancias, por lo que se aumentan sus pesos. Por lo tanto, el segundo clasificador hace un mejor trabajo en estos casos, y así su aserto. 

El gráfico de la derecha representa la misma secuencia de predictores, excepto que la tasa de aprendizaje se reduce a la mitad (es decir, los pesos de la instancia mal clasificados se aumentan mucho menos en cada iteración). Como puede ver, esta técnica de aprendizaje secuencial tiene algunas similitudes con el descenso de gradiente, excepto que en lugar de ajustar los parámetros de un solo predictor para minimizar una función de costo, AdaBoost agrega predictores al conjunto, haciéndolo gradualmente mejor.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0707.png)

(_Figura 7-7. Entrenamiento secuencial AdaBoost con actualizaciones de peso de instancia_)

Una vez que se entrenan todos los predictores, el conjunto hace predicciones muy parecidos a embolsar o pegar, excepto que los predictores tienen diferentes pesos dependiendo de su precisión general en el conjunto de entrenamiento ponderado.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0708.png)

(_Figura 7-8. Límites de decisión de los predictores consecutivos_)


#### ADVERTENCIA:

Hay un inconveniente importante en esta técnica de aprendizaje secuencial: el entrenamiento no se puede paralelizar, ya que cada predictor solo se puede entrenar después de que el predictor anterior haya sido entrenado y evaluado. Como resultado, no escala tan bien como embolsado o pegado.

##### ---

Echemos un vistazo más de cerca al algoritmo AdaBoost. El peso de cada instancia **w(i)** se establece inicialmente en **1/m**. Se entrena un primer predictor, y su tasa de error ponderada **r1** se calcula en el conjunto de entrenamiento; véase la ecuación 7-1.

#### Ecuación 7-1. Tasa de error ponderada del predictor jth

<a href="https://ibb.co/Fb6jvLz"><img src="https://i.ibb.co/fYMPRzH/Captura-de-pantalla-2023-10-25-a-las-18-21-26.png" alt="Captura-de-pantalla-2023-10-25-a-las-18-21-26" border="0"></a>


El peso **αj** del predictor se calcula utilizando la ecuación 7-2, donde **η** es el hiperparámetro de la tasa de aprendizaje (por defecto es 1).⁠ 

Cuanto más preciso sea el predictor, mayor será su peso. Si solo está adivinando al azar, entonces su peso estará cerca de cero. Sin embargo, si la mayoría de las veces es incorrecto (es decir, menos preciso que la suposición aleatoria), entonces su peso será negativo.

#### Ecuación 7-2. Peso predictor

<a href="https://imgbb.com/"><img src="https://i.ibb.co/vY2H8nf/Captura-de-pantalla-2023-10-25-a-las-18-22-40.png" alt="Captura-de-pantalla-2023-10-25-a-las-18-22-40" border="0"></a>

A continuación, el algoritmo AdaBoost actualiza los pesos de la instancia, utilizando la Ecuación 7-3, que aumenta los pesos de las instancias mal clasificadas.

#### Ecuación 7-3. Regla de actualización de peso

<a href="https://imgbb.com/"><img src="https://i.ibb.co/C5VPRsn/Captura-de-pantalla-2023-10-25-a-las-18-23-28.png" alt="Captura-de-pantalla-2023-10-25-a-las-18-23-28" border="0"></a>

Luego, todos los pesos de la instancia se normalizan (es decir, divididos por SUM(i=1 to m)(w^i).

Finalmente, se entrena un nuevo predictor utilizando los pesos actualizados, y se repite todo el proceso: se calcula el peso del nuevo predictor, se actualizan los pesos de la instancia, luego se entrena otro predictor, y así suce. El algoritmo se detiene cuando se alcanza el número deseado de predictores, o cuando se encuentra un predictor perfecto.

Para hacer predicciones, AdaBoost simplemente calcula las predicciones de todos los predictores y las pesa utilizando los pesos del predictor **αj**. La clase predicha es la que recibe la mayoría de los votos ponderados (ver ecuación 7-4).

#### Ecuación 7-4. Predicciones de AdaBoost

<a href="https://imgbb.com/"><img src="https://i.ibb.co/nBrVnr5/Captura-de-pantalla-2023-10-25-a-las-18-25-32.png" alt="Captura-de-pantalla-2023-10-25-a-las-18-25-32" border="0"></a>

Scikit-Learn utiliza una versión multiclase de AdaBoost llamada **SAMME⁠** (que significa Stagewise Additive Modeling utilizando una función de pérdida exponencial multiclase). 

Cuando solo hay dos clases, SAMME es equivalente a AdaBoost. Si los predictores pueden estimar las probabilidades de clase (es decir, si tienen un método predict_proba()), Scikit-Learn puede usar una variante de SAMME llamada SAMME.R (la R significa "Real"), que se basa en las probabilidades de clase en lugar de en las predicciones y generalmente funciona mejor.

El siguiente código entrena un clasificador AdaBoost basado en 30 stumps de decisión utilizando la clase `AdaBoostClassifier` de Scikit-Learn (como es de esperar, también hay una clase `AdaBoostRegressor`). Un muñón de decisión es un árbol de decisión con `max_depth=1`; en otras palabras, un árbol compuesto por un único nodo de decisión más dos nodos hoja. Este es el estimador base predeterminado para la clase `AdaBoostClassifier`:

In [36]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=30,
    learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=30, random_state=42)

##### TIP

Si su conjunto AdaBoost está sobreadaptando el conjunto de entrenamiento, puede intentar reducir el número de estimadores o regularizar más fuertemente el estimador base.

##### ----

## Aumento del gradiente

Otro algoritmo de refuerzo muy popular es el aumento de gradiente.⁠17 Al igual que AdaBoost, el aumento de gradiente funciona añadiendo secuencialmente predictores a un conjunto, cada uno corrigiendo a su predecesor. Sin embargo, en lugar de ajustar los pesos de la instancia en cada iteración como lo hace AdaBoost, este método intenta ajustar el nuevo predictor a los errores residuales cometidos por el predictor anterior.

Pasemos por un ejemplo simple de regresión, utilizando árboles de decisión como predictores de base; esto se llama aumento de árbol de gradiente, o árboles de regresión impulsados de gradiente (GBRT). Primero, generemos un conjunto de datos cuadrático ruidoso y ajustemos a DecisionTreeRegressor:

In [37]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3 * X[:, 0] ** 2 + 0.05 * np.random.randn(100)  # y = 3x² + Gaussian noise

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2, random_state=42)

A continuación, entrenaremos un segundo DecisionTreeRegressor sobre los errores residuales cometidos por el primer predictor:

In [38]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=43)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2, random_state=43)

Y luego entrenaremos a una tercera regresión en los errores residuales cometidos por el segundo predictor:

In [39]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=44)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2, random_state=44)

Ahora tenemos un conjunto que contiene tres árboles. Puede hacer predicciones sobre una nueva instancia simplemente sumando las predicciones de todos los árboles:

In [40]:
X_new = np.array([[-0.4], [0.], [0.5]])
sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

array([0.49484029, 0.04021166, 0.75026781])

La figura 7-9 representa las predicciones de estos tres árboles en la columna de la izquierda, y las predicciones del conjunto en la columna de la derecha. En la primera fila, el conjunto tiene solo un árbol, por lo que sus predicciones son exactamente las mismas que las del primer árbol. En la segunda fila, se entrena un nuevo árbol sobre los errores residuales del primer árbol. A la derecha se puede ver que las predicciones del conjunto son iguales a la suma de las predicciones de los dos primeros árboles. Del mismo modo, en la tercera fila se entrena otro árbol sobre los errores residuales del segundo árbol. Puedes ver que las predicciones del conjunto mejoran gradualmente a medida que se añaden árboles al conjunto.

Puede utilizar la clase `GradientBoostingRegressor` de Scikit-Learn para entrenar conjuntos GBRT más fácilmente (también hay una clase `GradientBoostingClassifier` para clasificación). Al igual que la clase `RandomForestRegressor`, tiene hiperparámetros para controlar el crecimiento de los árboles de decisión (por ejemplo, `max_depth`, `min_samples_leaf`), así como hiperparámetros para controlar el entrenamiento del conjunto, como el número de árboles (`n_estimators`). El siguiente código crea el mismo conjunto que el anterior:

In [41]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3,
                                 learning_rate=1.0, random_state=42)
gbrt.fit(X, y)


GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=42)

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0709.png)

(_Figura 7-9. En esta representación del aumento del gradiente, el primer predictor (arriba a la izquierda) se entrena normalmente, luego cada predictor consecutivo (centro izquierdo e inferior izquierda) se entrena en los residuos del predictor anterior; la columna de la derecha muestra las predicciones del conjunto resultante_)

El hiperparámetro `learning_rate` escala la contribución de cada árbol. Si lo establece en un valor bajo, como `0,05`, necesitará más árboles en el conjunto para ajustarse al conjunto de entrenamiento, pero las predicciones normalmente se generalizarán mejor. Esta es una técnica de regularización llamada **shrinkage**. 
La Figura 7-10 muestra dos conjuntos de GBRT entrenados con diferentes hiperparámetros: el de la izquierda no tiene suficientes árboles para adaptarse al conjunto de entrenamiento, mientras que el de la derecha tiene aproximadamente la cantidad correcta. Si agregamos más árboles, el GBRT comenzaría a sobreajustar el conjunto de entrenamiento.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0710.png)

(_Figura 7-10. Conjuntos GBRT con no suficientes predictores (izquierda) y suficientes (derecha)_)

Para encontrar la cantidad óptima de árboles, puede realizar una validación cruzada usando `GridSearchCV` o `RandomizedSearchCV`, como de costumbre, pero hay una forma más sencilla: si configura el hiperparámetro `n_iter_no_change` en un valor entero, digamos 10, entonces el `GradientBoostingRegressor` dejará automáticamente de agregar más. árboles durante el entrenamiento si ve que los últimos 10 árboles no ayudaron. Esto es simplemente una parada temprana (introducida en el Capítulo 4), pero con un poco de paciencia: tolera no tener progreso durante algunas iteraciones antes de detenerse. Entrenemos al conjunto usando paradas tempranas:

In [42]:
gbrt_best = GradientBoostingRegressor(
    max_depth=2, learning_rate=0.05, n_estimators=500,
    n_iter_no_change=10, random_state=42)
gbrt_best.fit(X, y)

GradientBoostingRegressor(learning_rate=0.05, max_depth=2, n_estimators=500,
                          n_iter_no_change=10, random_state=42)

Si establece `n_iter_no_change` demasiado bajo, el entrenamiento puede detenerse demasiado pronto y el modelo no se ajustará bien. Pero si lo configuras demasiado alto, se ajustará demasiado. También establecimos una tasa de aprendizaje bastante pequeña y una gran cantidad de estimadores, pero la cantidad real de estimadores en el conjunto entrenado es mucho menor, gracias a la detención temprana:

In [43]:
gbrt_best.n_estimators_

92

Cuando se establece `n_iter_no_change`, el método `fit()` divide automáticamente el conjunto de entrenamiento en un conjunto de entrenamiento más pequeño y un conjunto de validación: esto le permite evaluar el rendimiento del modelo cada vez que agrega un nuevo árbol. El tamaño del conjunto de validación está controlado por el hiperparámetro `validation_fraction`, que es del 10% de forma predeterminada. El hiperparámetro `tol` determina la mejora máxima del rendimiento que aún se considera insignificante. El valor predeterminado es 0,0001.

La clase `GradientBoostingRegressor` también admite un hiperparámetro de `subsample`, que especifica la fracción de instancias de entrenamiento que se utilizarán para entrenar cada árbol. Por ejemplo, si `subsample = 0,25`, entonces cada árbol se entrena en el 25 % de las instancias de entrenamiento, seleccionadas al azar. Como probablemente ya habrás adivinado, esta técnica intercambia un sesgo más alto por una varianza más baja. También acelera considerablemente el entrenamiento. Esto se llama aumento de gradiente estocástico.


## Aumento del gradiente basado en histogramas

Scikit-Learn también proporciona otra implementación de GBRT, optimizada para grandes conjuntos de datos: aumento de gradiente basado en histogramas (HGB). Funciona bininging las características de entrada, reemplazándolas con números enteros. El número de contenedores está controlado por el hiperparámetro themax `max_bins`, que por defecto es 255 y no se puede establecer más alto que esto. Binning puede reducir en gran medida el número de posibles umbrales que el algoritmo de entrenamiento necesita evaluar. Además, trabajar con números enteros permite utilizar estructuras de datos más rápidas y eficientes en la memoria. Y la forma en que se construyen los contenedores elimina la necesidad de clasificar las características al entrenar cada árbol.

Como resultado, esta implementación tiene una complejidad computacional de O(b×m) en lugar de O(n×m×log(m)), donde b es el número de contenedores, m es el número de instancias de entrenamiento y n es el número de características. En la práctica, esto significa que HGB puede entrenar cientos de veces más rápido que el GBRT normal en grandes conjuntos de datos. Sin embargo, el binning causa una pérdida de precisión, que actúa como un regularizador: dependiendo del conjunto de datos, esto puede ayudar a reducir el ajuste excesivo, o puede causar un ajuste insuficiente.

Scikit-Learn proporciona dos clases para HGB: `HistGradientBoostingRegressor` y `HistGradientBoostingClassifier`. Son similares a `GradientBoostingRegressor` y `GradientBoostingClassifier`, con algunas diferencias notables:

- La parada anticipada se activa automáticamente si el número de instancias es superior a 10.000. Puede activar o desactivar siempre la parada anticipada estableciendo el hiperparámetro `early_stopping` en `True` o `False`.

- El submuestreo no es compatible.

- `n_estimators` cambia de nombre a `max_iter`.

- Los únicos hiperparámetros del árbol de decisión que se pueden modificar son `max_leaf_nodes`,  `min_samples_leaf` y `max_ depth`.

Las clases HGB también tienen dos características interesantes: admiten tanto características categóricas como valores faltantes. Esto simplifica bastante el preprocesamiento. Sin embargo, las características categóricas deben representarse como números enteros que van desde 0 hasta un número inferior a `max_bins`. Puede utilizar un `OrdinalEncoder` para esto. Por ejemplo, aquí se explica cómo construir y entrenar un proceso completo para el conjunto de datos de vivienda de California presentado en el Capítulo 2:

In [44]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder

hgb_reg = make_pipeline(
    make_column_transformer((OrdinalEncoder(), ["ocean_proximity"]),
                            remainder="passthrough"),
    HistGradientBoostingRegressor(categorical_features=[0], random_state=42)
)
hgb_reg.fit(housing, housing_labels)

NameError: name 'housing' is not defined

¡Todo el proceso es tan corto como las importaciones! No se necesita una computadora, un escalador o un codificador one-hot, por lo que es realmente conveniente. Tenga en cuenta que `categorical_features` debe establecerse en los índices de columnas categóricas (o una matriz booleana). Sin ningún ajuste de hiperparámetros, este modelo produce un RMSE de aproximadamente 47.600, lo cual no está tan mal.

##### TIP

Varias otras implementaciones optimizadas de aumento de gradiente están disponibles en el ecosistema de Python ML: en particular, XGBoost, CatBoost y LightGBM. Estas bibliotecas han existido durante varios años. Todos están especializados para aumentar el gradiente, sus API son muy similares a las de Scikit-Learn y proporcionan muchas características adicionales, incluida la aceleración de la GPU; ¡definitivamente deberías echarles un vistazo! Además, la biblioteca de Bosques Aleatorios de TensFlow proporciona implementaciones optimizadas de una variedad de algoritmos de bosques aleatorios, incluidos bosques aleatorios simples, extraárboles, GBRT y varios más.

##### ---

## Apilado

El último método de conjunto que discutiremos en este capítulo se llama apilamiento (abreviatura de generalización apilada).⁠ Se basa en una idea simple: en lugar de usar funciones triviales (como la votación dura) para agregar las predicciones de todos los predictores en un conjunto, ¿por qué no entrenamos a un modelo para realizar esta agregación? La figura 7-11 muestra a un conjunto de este tipo realizando una tarea de regresión en una nueva instancia. Cada uno de los tres predictores inferiores predice un valor diferente (3,3, 2,7 y 2,9), y luego el predictor final (llamado licuadora o metaaprendiz) toma estas predicciones como entradas y hace la predicción final (3,3.0).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0711.png)

(_Figura 7-11. Agregación de predicciones utilizando un predictor de mezcla_)

Para entrenar la licuadora, primero debe crear el conjunto de entrenamiento de mezcla. Puede usar `cross_val_predict()` en cada predictor del conjunto para obtener predicciones fuera de la muestra para cada instancia en el conjunto de entrenamiento original (Figura 7-12), y usarlas como características de entrada para entrenar el mezclador; y los objetivos pueden simplemente copiarse del conjunto de entrenamiento original. Tenga en cuenta que, independientemente de la cantidad de características en el conjunto de entrenamiento original (solo una en este ejemplo), el conjunto de entrenamiento combinado contendrá una característica de entrada por predictor (tres en este ejemplo). Una vez que se entrena el mezclador, los predictores base se vuelven a entrenar una última vez en el conjunto de entrenamiento original completo.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0712.png)

(_Figura 7-12. Entrenando el blender en un conjunto de apilamiento_)

En realidad, es posible entrenar varias licuadoras diferentes de esta manera (por ejemplo, una usando regresión lineal, otra usando regresión forestal aleatoria) para obtener una capa completa de licuadoras, y luego agregar otra licuadora encima de eso para producir la predicción final, como se muestra en la Figura 7-13. Es posible que pueda exprimir unas cuantas gotas más de rendimiento haciendo esto, pero le costará tanto en el tiempo de entrenamiento como en la complejidad del sistema.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0713.png)

(_Figura 7-13. Predicciones en un conjunto de apilamiento multicapa_)

Scikit-Learn proporciona dos clases para apilar conjuntos: `StackingClassifier` y `StackingRegressor`. Por ejemplo, podemos reemplazar el `VotingClassifier` que usamos al comienzo de este capítulo en el conjunto de datos de las lunas con un `StackingClassifier`:

In [45]:
from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(probability=True, random_state=42))
    ],
    final_estimator=RandomForestClassifier(random_state=43),
    cv=5  # number of cross-validation folds
)
stacking_clf.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression(random_state=42)),
                               ('rf', RandomForestClassifier(random_state=42)),
                               ('svc', SVC(probability=True, random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=43))

Para cada predictor, el clasificador de apilamiento llamará a `predict_proba()` si está disponible; de lo contrario, recurrirá a `decision_function()` o, como último recurso, llamará a `predict()`. Si no proporciona un estimador final, `StackingClassifier` usará `LogisticRegression` y `StackingRegressor` usará RidgeCV.

Si evalúa este modelo de apilamiento en el conjunto de pruebas, encontrará una precisión del 92,8 %, que es un poco mejor que el clasificador de votación que utiliza la votación suave, que obtuvo el 92 %.

En conclusión, los métodos de conjunto son versátiles, potentes y bastante fáciles de usar. Los bosques aleatorios, AdaBoost y GBRT se encuentran entre los primeros modelos que debe probar para la mayoría de las tareas de aprendizaje automático, y brillan particularmente con datos tabulares heterogéneos. Además, como requieren muy poco preprocesamiento, son excelentes para poner en marcha un prototipo rápidamente. Por último, los métodos de conjunto como los clasificadores de votación y los clasificadores de apilamiento pueden ayudar a llevar el rendimiento de su sistema a sus límites.



# Ejercicios


- Si has entrenado cinco modelos diferentes con exactamente los mismos datos de entrenamiento, y todos logran una precisión del 95 %, ¿hay alguna posibilidad de que puedas combinar estos modelos para obtener mejores resultados? Si es así, ¿cómo? Si no, ¿por qué?

- ¿Cuál es la diferencia entre los clasificadores de votación dura y blanda?

- ¿Es posible acelerar el entrenamiento de un conjunto de embolsado distribuyéndolo a través de múltiples servidores? ¿Qué pasa con los conjuntos de pegado, los conjuntos de impulso, los bosques aleatorios o los conjuntos de apilamiento?

- ¿Cuál es el beneficio de la evaluación fuera de la bolsa?

- ¿Qué hace que los conjuntos de extraárboles sean más aleatorios que los bosques aleatorios normales? ¿Cómo puede ayudar esta aleatoriedad adicional? ¿Los clasificadores de extraárboles son más lentos o más rápidos que los bosques aleatorios normales?

- Si su conjunto AdaBoost no se ajusta a los datos de entrenamiento, ¿qué hiperparámetros debe ajustar y cómo?

- Si su conjunto de aumento de gradiente se adapta demasiado al conjunto de entrenamiento, ¿debería aumentar o disminuir la tasa de aprendizaje?

- Cargue el conjunto de datos MNIST (introducido en el Capítulo 3) y divídalo en un conjunto de entrenamiento, un conjunto de validación y un conjunto de pruebas (por ejemplo, use 50.000 instancias para el entrenamiento, 10.000 para la validación y 10.000 para las pruebas). Luego entrena a varios clasificadores, como un clasificador de bosques aleatorio, un clasificador de árboles adicionales y un clasificador SVM. A continuación, intente combinarlos en un conjunto que supere a cada clasificador individual en el conjunto de validación, utilizando una votación suave o dura. Una vez que hayas encontrado uno, pruébalo en el conjunto de pruebas. ¿Cuánto mejor funciona en comparación con los clasificadores individuales?

- Run the individual classifiers from the previous exercise to make predictions on the validation set, and create a new training set with the resulting predictions: each training instance is a vector containing the set of predictions from all your classifiers for an image, and the target is the image’s class. Train a classifier on this new training set. Congratulations—you have just trained a blender, and together with the classifiers it forms a stacking ensemble! Now evaluate the ensemble on the test set. For each image in the test set, make predictions with all your classifiers, then feed the predictions to the blender to get the ensemble’s predictions. How does it compare to the voting classifier you trained earlier? Now try again using a StackingClassifier instead. Do you get better performance? If so, why?

- Las soluciones a estos ejercicios están disponibles al final del cuaderno de este capítulo, en https://homl.info/colab3.

